In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import sqlalchemy
from sqlalchemy import select, text
import xmltodict
from lxml import etree
from cytoolz import get_in
from itertools import islice
from datetime import datetime

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.io as cio
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.primers as primers
import paulssonlab.api.geneious as geneious
import paulssonlab.api.geneious_db as g

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
geneious_sessionmaker = geneious.connect(**config["geneious_test"])

In [ ]:
reg = registry.Registry(
    gc,
    config["registry"]["folder"],
    geneious_sessionmaker=geneious_sessionmaker,
    geneious_folder=("registry", "test1"),
)

# Sync

In [ ]:
import re


def strip_leading_lowercase(s):
    return re.sub(r"^[a-z]*", "", s)


def ensure_folder(path, root):
    folder = root
    for key in path:
        print(key, path)
        if key in folder:
            folder = folder[key]
        else:
            # session.add(g.Folder(name=key, parent_folder=folder, g_group_id=folder.g_group_id))
            new_folder = g.Folder(name=key, g_group_id=folder.g_group_id)
            folder.children[key] = new_folder
            folder = new_folder
    return folder


def update_mtimes(folder):
    while folder is not None:
        folder.modified = text("current_timestamp")
        folder = folder.parent_folder

In [ ]:
session = reg.geneious_sessionmaker()

In [ ]:
# g.Base.metadata.drop_all(session.bind)

In [ ]:
root_folder = session.execute(
    select(g.Folder).where(g.Folder.parent_folder_id == None)
).one()[0]

In [ ]:
geneious_registry_folder = ensure_folder(reg.geneious_folder, root_folder)

In [ ]:
# user = session.execute(select(g.GUser).where(g.GUser.username == "remoteuser")).one()[0]

In [ ]:
for key in reg:
    type_ = key[1]
    print(key)
    if key[0].endswith("LIB") and key[1] in ("plasmids",):
        client = reg[key]
        subfolder_key = (strip_leading_lowercase(key[0]), "_".join(key))
        collection_folder = ensure_folder(subfolder_key, geneious_registry_folder)
        for name in islice(client.keys(), 20, 30):
            entry = reg.get_with_prefix(name, key[0], type_)
            print(entry)
            if type_ == "oligos":
                document_type = "primer"
                name = entry["_id"]
                description = entry["Name"]
            elif type_ in ("parts", "plasmids"):
                document_type = "genbank"
                name = entry["_id"]
                description = entry["Description"]
            else:
                raise NotImplementedError
            seq = entry["_seq"]
            urn = geneious.make_urn()
            # plugin_urn = geneious.make_urn()
            timestamp = geneious.geneious_timestamp()
            dt = datetime.fromtimestamp(timestamp / 1000)
            # print(name, description)
            document_xml, plugin_document_xml = geneious.dumps_geneious(
                seq,
                urn,
                document_type=document_type,
                name=name,
                description=description,
                timestamp=timestamp,
            )
            doc = g.AnnotatedDocument(
                parent_folder=collection_folder,
                urn=urn,
                modified=dt,
                document_xml=document_xml,
                plugin_document_xml=plugin_document_xml,
                reference_count=0,
            )
            session.add(doc)
        # TODO
        collection_folder.modified = text("current_timestamp")
        # update_mtimes(collection_folder)

In [ ]:
collection_folder.modified = text("current_timestamp")

In [ ]:
session.commit()

In [ ]:
session.execute(select(g.Folder)).all()

In [ ]:
session.execute(select(g.AnnotatedDocument)).all()[-1][0].modified = text(
    "current_timestamp"
)

In [ ]:
session.execute(select(g.Folder)).all()[-1][0].modified = text("current_timestamp")

In [ ]:
session.commit()

In [ ]:
session.execute(select(g.NextTableId)).all()